In [1]:
import pandas as pd
import numpy as np
import json
import collections
import re

In [2]:
file_names = {
    "initial": [
        "Commcare INITIAL Survey updated w Renewvia Numbers - Copy.xlsx",
        "MS Form - INITIAL SURVEY Updated w meter numbers 230216.xlsx"
    ],
    
    "hs_post": [
        "MS Form - POST Connection HOUSEHOLD.xlsx", 
        "Commcare POST Connection HOUSEHOLD.xlsx"
    ],
    
    "ci_post": [
        "MS Form - POST Connection - COMMERCIAL & INSTITUTION Connection Impact Survey.xlsx",
        "Commcare - POST Connection - COMMERCIAL & INSTITUTION Connectin Impact Survey.xlsx"
    ],
    }

# Pulled SQL data for each customer id and their tariffs
tariffs = pd.read_csv("customers_tariffs.csv")

# mapping missing or mistyped answers
with open('col_answers_mapping.json', 'r') as f:
    col_answers_map = json.load(f)
    
answers = col_answers_map['col_answers_map']
n_col = len(answers)
        
for name, files in file_names.items():
    data_sets = []
    for file in files:
        df = pd.read_excel("datasets_raw/"+file)
        # df = datasets["hs_post_msform"]
        cols = list(df.columns)
        cols_clean = [col.replace('\xa0', '') if '\xa0' in col else col for col in cols]
        for i in range(0, n_col):
            old = answers[i]['old_name']
            if old in cols_clean:
                new = answers[i]['new_name']
                mapping = answers[i]['mapping']
                df = df.rename(columns={old:new})
                df[new].replace(mapping, inplace=True)
                special_cases = ["girls_age", "boys_age", 
                                 "minigrid_signup_primary_reason",
                                 "appliances_type"
                                 "appliances_type_addition"]

                if new in special_cases:
                    def replace_map(value, str_map):
                        if type(value) == str and "choice" in value:
                            value = value.replace(" ", ";")
                            parts, replaced = value.split(";"), []
                            cases = [ "cell phone charge", "cell phone charger",
                                    "cell_phone_charge"]
                            for sub in parts:
                                sub = sub.lower()
                                if sub == "older than 16":
                                    replaced.append("older_than_16")
                                elif sub in cases:
                                    replaced.append("cell_phone_charger")
                                else:
                                    replaced.append(str_map[sub])
                            return ";".join(replaced)
                        else:
                            return value

                    df[new] = df[new].map(lambda x: replace_map(x, mapping))
                # elif new == :
                elif new == "age":
                    df[new] = df[new].replace(["00", "000", "years", "_hours", " "], "")

            else:
                continue

        missing = ['         ', '    ', '   ',
                   '******', 'months', 'nan']
        df['renewvia_id'] = df['renewvia_id'].apply(lambda x: str(x))
        df['renewvia_id'].replace(missing, np.nan, inplace=True)
        df['end'] = pd.to_datetime(df['end'])
        df['end_date'] = df['end'].dt.date
        df_merged = df.merge(tariffs, left_on='renewvia_id',
                            right_on='customerAccountNumber', how='left')

        data_sets.append(df_merged)
        
    datasets_merged = pd.concat(data_sets, axis=0)
    datasets_merged.to_csv("datasets_annotated/"+name+"_annotated.csv")